# Persona API Configuration

In [1]:
# Import Libraries
import requests
import json
import pickle
import boto3
import pandas as pd
import math
from botocore.exceptions import ClientError
from pprint import pprint
import time
from IPython.display import display
from utils.functions import get_secret, refresh_access_token

## API Configurations

### Extracting credentials from Secrets Manager (recommended)

In [ ]:
# Option 1:Secrets Manager
secret_name = "prod/my_console/APIGW-client"
region_name = "us-east-1"
    
secret_cred = get_secret(secret_name = secret_name, region_name=region_name)

### Extracting credentials from local (optional)
In case you are not comfortable using a Secrets Manager, you could save your credentials locally and retrieve them.

In [ ]:
# Option 2:Local File (uncomment the line below and execute it)
# secret_cred = get_secret(secret_name = secret_name, region_name=region_name, source='local')

In [5]:
# Extracting the credentials from the file
Clientid = secret_cred['Clientid']
Clientsecret = secret_cred['Clientsecret']
returnurl = secret_cred['returnurl']
refresh_token = secret_cred['refreshToken']

### Instance information

In [ ]:
# Overlapping audiences playbook
instanceid = 'instance01'
#entityId = ''#'ENTITY1AA1AA11AAA1'


### URL endpoints

In [7]:
# AMC API URL
url = 'https://advertising-api.amazon.com'

# List AMC account
accountsUrl = url + "/amc/accounts"

### One-time steps - not required after refresh token is securely stored

In [ ]:
# Generate grant code and paste into a new tab
urlgrant = f'https://www.amazon.com/ap/oa?client_id={Clientid}&scope=advertising::campaign_management&advertising::audiences&response_type=code&redirect_uri={returnurl}'
# urlgrant = f'https://www.amazon.com/ap/oa?client_id={Clientid}&scope=advertising::audiences&response_type=code&redirect_uri={returnurl}'

display(urlgrant)

### Refresh access token at start of every session

In [ ]:
# Refresh access token for each session - refresh token valid for 1 hour
access_token =  refresh_access_token(url = 'https://api.amazon.com/auth/o2/token', 
                     Clientid=Clientid,Clientsecret=Clientsecret, 
                     refresh_token=refresh_token) 

headers = {
        'Authorization': 'Bearer ' + access_token,
        'Amazon-Advertising-API-ClientId': Clientid,
        #'Amazon-Advertising-API-AdvertiserId': entityId,
        #'Amazon-Advertising-API-MarketplaceId': 'ATVPDKIKX0DER'
       }

### Access your account

In [ ]:
# Accounts
accounts = requests.get(accountsUrl, headers = headers)

#print(accounts.text)

# Extracting AccountID & MarketplaceID
if accounts.ok:
    # Select the second element from accounts - the id is needed for our use case
    accountId = accounts.json()['amcAccounts'][1]['accountId']
    marketplaceId = accounts.json()['amcAccounts'][1]['marketplaceId']
    
    print(f"accountId : {accountId}, \nmarketplaceId : {marketplaceId}")

### Access your profile

In [ ]:
# Function to access profiles
def get_profiles(client_id, access_token, url):
    try:
        headers = {
            'content-type': 'application/json',
            'Amazon-Advertising-API-ClientId': Clientid,
            'Authorization': 'Bearer ' + access_token
        }

        response = requests.get(url + '/v2/profiles', headers=headers)
        result = response.json()

    except Exception as e:
        print(str(e))

    return result


def get_profile(client_id, access_token, url, lstIds):
    lst_profile_info = []
    headers = {
        'content-type': 'application/json',
        'Amazon-Advertising-API-ClientId': Clientid,
        'Authorization': 'Bearer ' + access_token
    }

    for id in lstIds:
        response = requests.get(url + '/v2/profiles/' + id, headers=headers)
        result = response.json()
        lst_profile_info.append(result)

    return lst_profile_info